In [1]:
import warnings
from collections import defaultdict
from itertools import chain, product
from pathlib import Path

import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

tf.logging.set_verbosity(tf.logging.ERROR)
warnings.filterwarnings('ignore')

In [2]:
fpath = Path('../data/prepared_data.csv')

In [3]:
data = pd.read_csv(fpath, index_col=0)

In [4]:
data.head(10)

,customer_id,mccs,mccs_count,transaction_month
0,0001f322716470bf9bfc1708f06f00fc,"6011,6011,6011,6011,6011,6011,6011",7,2
1,0001f322716470bf9bfc1708f06f00fc,"6011,6011,6011,6011,6011,6011,6011",7,3
2,0001f322716470bf9bfc1708f06f00fc,"5541,6011,6011,6011,6011,6011,6011,6011,6011",9,4
3,0001f322716470bf9bfc1708f06f00fc,"5411,5411,5541,5912,6011,6011,6011,6011,6011",9,5
4,0001f322716470bf9bfc1708f06f00fc,"5411,5411,5411,5411,5499,5541,5541,5999,5999,6...",18,6
5,0001f322716470bf9bfc1708f06f00fc,"5411,5411,5541,5999,6011,6011,6011",7,7
6,0001f322716470bf9bfc1708f06f00fc,"5411,5411,6011,6011,6011,6011,6011",7,8
7,0001f322716470bf9bfc1708f06f00fc,"5211,5411,5541,6011,6011,6011,6011,6011,6011,6...",11,9
8,0007297d86e14bd68bd87b1dbdefe302,"6011,6011,6011,6011,6011,6011",6,2
9,0007297d86e14bd68bd87b1dbdefe302,"6011,6011,6011,6011,6011,6011",6,3


В столбце `mccs` в `data` записан список всех категорий товаров, приобретённых в данном месяце, разделённых запятой.

In [5]:
def deserialize_int_list(list_string):
    return [int(item) for item in list_string.split(',')]

In [6]:
baskets = [
    deserialize_int_list(list_string)
    for list_string in data['mccs']
]

In [7]:
all_items = list(chain.from_iterable(baskets))

unique_items = sorted(list(set(all_items)))

In [8]:
unique_customers = data['customer_id'].unique().tolist()

Получаем векторы корзин с помощью Bag of words.

In [9]:
def encode_basket(basket, vocabulary):
    vector = np.zeros(len(vocabulary))
    
    for item in basket:
        item_id = vocabulary.index(item)
        
        vector[item_id] += 1
    
    return vector

In [10]:
encoded_baskets = np.array([encode_basket(basket, unique_items) for basket in baskets])

Масштабируем значения в векторах:

* логарифмируем
* располагаем между 0 и 1

In [11]:
encoded_baskets = np.log2(encoded_baskets + 1)

In [12]:
scaler = MinMaxScaler()

encoded_baskets = scaler.fit_transform(encoded_baskets)

Следующая функция получает на вход длину сиквенса `sequence_length` и возвращает обучающую и тестовую выборки для данной длины. Для каждого пользователя:

1. собираем все корзины.
2. получаем из них всевозможные последовательности корзин длины `sequence_length`, причём, если в качестве `x` мы берём слайс `baskets[i:j]`, то в качестве соответствующего `y` выступит `baskets[i+1 : j+1]`.
3. все `y` бинаризируем по правилу `y = (y > 0).astype(int)`.
4. все такие последовательности кроме последней (для которой `y` содержит самую последнюю корзину) складываем в обучающую выборку, а последнюю - в тестовую.

In [13]:
customer2baskets = {
    customer_id: encoded_baskets[data['customer_id'] == customer_id]
    for customer_id in tqdm(unique_customers)
}

100%|██████████| 9988/9988 [00:22<00:00, 443.10it/s]


In [14]:
def get_train_and_test_data(sequence_length=3):
    x_train = list()
    y_train = list()

    x_test = list()
    y_test = list()

    for customer_id in unique_customers:
        customer_baskets = customer2baskets[customer_id]
        baskets_count = len(customer_baskets)

        sequence_count = baskets_count - sequence_length

        if sequence_count < 2:
            continue

        for start_id in range(sequence_count):
            end_id = start_id + sequence_length

            x = customer_baskets[start_id : end_id].copy()
            y = customer_baskets[start_id + 1 : end_id + 1].copy()
            y = (y > 0).astype(int)

            if start_id < sequence_count - 1:
                x_train.append(x)
                y_train.append(y)
            else:
                x_test.append(x)
                y_test.append(y)

    x_train = np.array(x_train)
    y_train = np.array(y_train)

    x_test = np.array(x_test)
    y_test = np.array(y_test)
    
    return x_train, x_test, y_train, y_test

Собираем последовательности всевозможной длины.

In [15]:
x_train = dict()
y_train = dict()

x_test = dict()
y_test = dict()

for sequence_length in range(1, 13):
    x_train_, x_test_, y_train_, y_test_ = get_train_and_test_data(sequence_length)
    
    if len(x_train_) == 0:
        break
    
    x_train[sequence_length] = x_train_
    y_train[sequence_length] = y_train_
    
    x_test[sequence_length] = x_test_
    y_test[sequence_length] = y_test_

In [18]:
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(
        512,
        input_shape=(None, len(unique_items)),
        return_sequences=True,
        dropout=0.2,
        recurrent_dropout=0.2,
    ),
    tf.keras.layers.Dense(
        512,
        activation='sigmoid',
    ),
    tf.keras.layers.Dense(
        len(unique_items),
        activation='sigmoid',
    ),
])

In [19]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
)

Последовательно обучаем модель на сиквенсах каждой длины.

In [20]:
epochs = 10

for epoch in range(epochs):
    print(f'Epoch {epoch + 1}.')
    
    for sequence_length in x_train:
        model.fit(
            x_train[sequence_length],
            y_train[sequence_length],
            verbose=1,
            validation_data=(x_test[sequence_length], y_test[sequence_length]),
        )
    
    print()

Epoch 1.
Train on 46501 samples, validate on 9617 samples
46501/46501 [==============================] - 36s 781us/sample - loss: 0.0511 - val_loss: 0.0458
Train on 36884 samples, validate on 9165 samples
36884/36884 [==============================] - 40s 1ms/sample - loss: 0.0482 - val_loss: 0.0460
Train on 27719 samples, validate on 8549 samples
27719/27719 [==============================] - 39s 1ms/sample - loss: 0.0477 - val_loss: 0.0468
Train on 19170 samples, validate on 7703 samples
19170/19170 [==============================] - 34s 2ms/sample - loss: 0.0472 - val_loss: 0.0470
Train on 11467 samples, validate on 6149 samples
11467/11467 [==============================] - 25s 2ms/sample - loss: 0.0464 - val_loss: 0.0462
Train on 5318 samples, validate on 4840 samples
5318/5318 [==============================] - 14s 3ms/sample - loss: 0.0415 - val_loss: 0.0555
Train on 478 samples, validate on 478 samples
478/478 [==============================] - 1s 3ms/sample - loss: 0.0392 - va

46501/46501 [==============================] - 38s 810us/sample - loss: 0.0430 - val_loss: 0.0443
Train on 36884 samples, validate on 9165 samples
36884/36884 [==============================] - 42s 1ms/sample - loss: 0.0425 - val_loss: 0.0447
Train on 27719 samples, validate on 8549 samples
27719/27719 [==============================] - 42s 2ms/sample - loss: 0.0409 - val_loss: 0.0433
Train on 19170 samples, validate on 7703 samples
19170/19170 [==============================] - 34s 2ms/sample - loss: 0.0391 - val_loss: 0.0412
Train on 11467 samples, validate on 6149 samples
11467/11467 [==============================] - 25s 2ms/sample - loss: 0.0382 - val_loss: 0.0361
Train on 5318 samples, validate on 4840 samples
5318/5318 [==============================] - 15s 3ms/sample - loss: 0.0333 - val_loss: 0.0470
Train on 478 samples, validate on 478 samples
478/478 [==============================] - 2s 3ms/sample - loss: 0.0299 - val_loss: 0.0355

Epoch 10.
Train on 46501 samples, validate

Проверяем результаты.

In [21]:
def f1(y_test, y_pred):
    f1s = list()
    
    for test, pred in zip(y_test, y_pred):
        test = test[-1]
        pred = pred[-1]
        
        f1_ = f1_score(test, (pred > 0.5).astype(int))
        
        f1s.append(f1_)
    
    return np.mean(f1s)

In [22]:
print('seq_len\tf1')

for sequence_length in x_test:
    y_pred = model.predict(x_test[sequence_length])
    
    print(f'{sequence_length}\t{f1(y_test[sequence_length], y_pred)}')

seq_len	f1
1	0.4981297462904875
2	0.5591226013089906
3	0.581323399599347
4	0.5971323636095175
5	0.6186631981501756
6	0.6379651289411066
7	0.6406057093879871


Топ-5 предсказанных категорий.

In [23]:
def get_nlargest_ids(array, top=5):
    return array.argsort()[:-top - 1:-1]

In [24]:
unique_items = np.array(unique_items)

In [28]:
for customer_id in np.random.choice(unique_customers, 5, replace=False):
    customer_baskets = encoded_baskets[data['customer_id'] == customer_id]

    next_basket_real = customer_baskets[-1]

    top_5_ids_real = get_nlargest_ids(next_basket_real)
    top_5_results_real = unique_items[top_5_ids_real]

    customer_baskets = customer_baskets[:-1]
    customer_baskets = customer_baskets[np.newaxis, :]
    
    next_baskets_pred = model.predict(customer_baskets)[0]
    next_basket_pred = next_baskets_pred[-1]

    top_5_ids_pred = get_nlargest_ids(next_basket_pred)
    top_5_results_pred = unique_items[top_5_ids_pred]
    
    top_5_results_real.sort()
    top_5_results_pred.sort()
    
    print(f'customer id: {customer_id}')
    
    print(f'real: {top_5_results_real}')
    print(f'pred: {top_5_results_pred}\n')

customer id: 874e4643199a9adcd65e85d733679c33
real: [5411 5541 5661 5812 5814]
pred: [5411 5541 5812 5814 6011]

customer id: 27dcbe0058941b092c03e05a360d55c5
real: [5074 5085 5411 5541 6011]
pred: [5411 5541 5814 5921 6011]

customer id: aa51500ce03cca814cc5f9f02551b242
real: [5198 5331 5411 5814 9402]
pred: [5411 5499 5541 5912 6011]

customer id: 1f6659c9a9cd28fc8e0f484ddf56d1cd
real: [5310 5311 5399 5411 5499]
pred: [5411 5812 5814 5912 6011]

customer id: ae18d7f44678a0f682575699e006162a
real: [5310 5311 5411 5691 6011]
pred: [5411 5499 5691 5912 6011]

